# Hyperparameter Tuning For Music Generation
## Import Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  
import pandas as pd
import time
from sklearn.metrics import accuracy_score

## Load and Prepare Data

In [2]:
df = pd.read_csv("biggerFileMusicInfo.csv")
df

,Unnamed: 0,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,spectral_entropy_mean,spectral_flux_mean,spectral_rolloff_mean,mfcc_1_mean,...,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std,Row_Index,Genre,GenreID
0,0,0.040872,0.020890,0.013624,0.009991,0.013624,0.026340,0.057221,0.177112,0.189827,...,0.008174,0.012716,0.050863,0.057221,0.029973,0.020890,0.009991,0.0,AmbientElectronic10,0.0
1,1,0.039055,0.060854,0.059037,0.041780,0.032698,0.037239,0.033606,0.029973,0.023615,...,0.079927,0.086285,0.073569,0.089010,0.102634,0.087193,0.082652,1.0,AmbientElectronic10,1.0
2,2,0.080836,0.071753,0.068120,0.064487,0.048138,0.041780,0.026340,0.019074,0.022707,...,0.037239,0.028156,0.024523,0.015441,0.009991,0.017257,0.040872,2.0,AmbientElectronic10,2.0
3,3,0.020890,0.015441,0.016349,0.019982,0.013624,0.008174,0.009083,0.009083,0.009083,...,0.018165,0.016349,0.022707,0.037239,0.031789,0.021798,0.019982,3.0,AmbientElectronic10,3.0
4,4,0.011807,0.018165,0.045413,0.073569,0.073569,0.070845,0.064487,0.066303,0.087193,...,0.013624,0.005450,0.007266,0.009991,0.007266,0.021798,0.025431,4.0,AmbientElectronic10,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9343,9343,0.106267,0.116258,0.113533,0.113533,0.089918,0.091735,0.096276,0.092643,0.084469,...,0.138965,0.129882,0.130790,0.126249,0.108992,0.107175,0.110808,9298.0,Hip-Hop4,9298.0
9344,9344,0.053588,0.060854,0.040872,0.037239,0.046322,0.049046,0.037239,0.031789,0.032698,...,0.087193,0.081744,0.098093,0.131698,0.138056,0.114441,0.084469,9299.0,Hip-Hop4,9299.0
9345,9345,0.087193,0.093551,0.124432,0.128065,0.133515,0.111717,0.093551,0.080836,0.087193,...,0.099001,0.095368,0.066303,0.071753,0.084469,0.079927,0.078111,9300.0,Hip-Hop4,9300.0
9346,9346,0.078111,0.055404,0.051771,0.059946,0.061762,0.043597,0.038147,0.082652,0.128065,...,0.122616,0.092643,0.067212,0.080836,0.095368,0.103542,0.110808,9301.0,Hip-Hop4,9301.0


In [3]:
df_train = (df.copy()).iloc[:, 1:-3]

In [4]:
df_train.head()

,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,spectral_entropy_mean,spectral_flux_mean,spectral_rolloff_mean,mfcc_1_mean,mfcc_2_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
0,0.040872,0.020890,0.013624,0.009991,0.013624,0.026340,0.057221,0.177112,0.189827,0.079019,...,0.009083,0.014532,0.013624,0.008174,0.012716,0.050863,0.057221,0.029973,0.020890,0.009991
1,0.039055,0.060854,0.059037,0.041780,0.032698,0.037239,0.033606,0.029973,0.023615,0.032698,...,0.088102,0.110808,0.091735,0.079927,0.086285,0.073569,0.089010,0.102634,0.087193,0.082652
2,0.080836,0.071753,0.068120,0.064487,0.048138,0.041780,0.026340,0.019074,0.022707,0.028156,...,0.008174,0.024523,0.041780,0.037239,0.028156,0.024523,0.015441,0.009991,0.017257,0.040872
3,0.020890,0.015441,0.016349,0.019982,0.013624,0.008174,0.009083,0.009083,0.009083,0.009991,...,0.038147,0.031789,0.026340,0.018165,0.016349,0.022707,0.037239,0.031789,0.021798,0.019982
4,0.011807,0.018165,0.045413,0.073569,0.073569,0.070845,0.064487,0.066303,0.087193,0.101726,...,0.012716,0.016349,0.019074,0.013624,0.005450,0.007266,0.009991,0.007266,0.021798,0.025431


In [5]:
X = df_train.values / 16
y = df.values[:, -2]

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y)

In [7]:
X_train.shape

(6543, 136)

In [8]:
y_train.shape

(6543,)

In [9]:
np.unique(y_train)

array(['AmbientElectronic10', 'Blues17', 'ChiptuneGlitch6', 'Classical11',
       'Country14', 'Electronic0', 'Experimental3', 'Folk5', 'Hip-Hop4'],
      dtype=object)

In [10]:
#(np.bincount(y_train) / len(y_train)).round(3)

In [11]:
X_train_orig = df_train.values[:, :-1]

In [12]:
np.unique(y_train)

array(['AmbientElectronic10', 'Blues17', 'ChiptuneGlitch6', 'Classical11',
       'Country14', 'Electronic0', 'Experimental3', 'Folk5', 'Hip-Hop4'],
      dtype=object)

In [13]:
y_train[:50].reshape(5, 10)

array([['Electronic0', 'AmbientElectronic10', 'Electronic0',
        'Electronic0', 'AmbientElectronic10', 'Experimental3', 'Folk5',
        'Electronic0', 'Electronic0', 'Classical11'],
       ['Experimental3', 'ChiptuneGlitch6', 'Experimental3',
        'Electronic0', 'Folk5', 'Classical11', 'Electronic0',
        'Electronic0', 'Electronic0', 'Electronic0'],
       ['Electronic0', 'ChiptuneGlitch6', 'ChiptuneGlitch6',
        'Electronic0', 'ChiptuneGlitch6', 'Experimental3', 'Electronic0',
        'Experimental3', 'Electronic0', 'ChiptuneGlitch6'],
       ['Electronic0', 'Electronic0', 'Experimental3',
        'AmbientElectronic10', 'Folk5', 'Folk5', 'AmbientElectronic10',
        'Experimental3', 'Folk5', 'Classical11'],
       ['Experimental3', 'Electronic0', 'Electronic0', 'Experimental3',
        'Electronic0', 'Experimental3', 'Experimental3', 'Electronic0',
        'Electronic0', 'Folk5']], dtype=object)

In [14]:
#df_test = pd.read_csv('optdigits.tes.csv', header=None)

In [15]:
#X_test = df_test.values[:, :-1] / 16
#y_test = df_test.values[:, -1]

In [16]:
X_test.shape

(2805, 136)

In [17]:
y_test.shape

(2805,)

## Decision Tree

In [18]:
from sklearn.tree import DecisionTreeClassifier

### Finding Best Parameter

In [19]:
depth_range = np.arange(5, 31)
criterion_range = ['gini', 'entropy']
dt_grid = {'max_depth': depth_range, 'criterion': criterion_range}
dt_grid

{'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
        22, 23, 24, 25, 26, 27, 28, 29, 30]),
 'criterion': ['gini', 'entropy']}

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
dt_gs = GridSearchCV(estimator=DecisionTreeClassifier(), 
                  param_grid=dt_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [22]:
current_time = time.time()
dt_gs.fit(X_train, y_train)
dt_gs_fit_time = time.time() - current_time

In [23]:
dt_gs.best_params_

{'criterion': 'gini', 'max_depth': 5}

In [24]:
dt_gs.best_score_

0.3336394941629696

In [25]:
best_dt = dt_gs.best_estimator_
best_dt

DecisionTreeClassifier(max_depth=5)

In [26]:
current_time = time.time()
y_train_predict_dt_gs = dt_gs.predict(X_train)
dt_gs_train_time = time.time() - current_time
dt_gs_train = accuracy_score(y_train, y_train_predict_dt_gs)
dt_gs_train

0.3403637475164298

In [27]:
current_time = time.time()
y_test_predict_dt_gs = dt_gs.predict(X_test)
dt_gs_test_time = time.time() - current_time
dt_gs_test = accuracy_score(y_test, y_test_predict_dt_gs)
dt_gs_test

0.3226381461675579

### Fitting, Training, Testing with Best Params

In [28]:
stump = DecisionTreeClassifier(criterion= 'gini', max_depth=dt_gs.best_params_["max_depth"])

In [29]:
current_time = time.time()
stump.fit(X_train, y_train)
stump_fit_time = time.time() - current_time

In [30]:
current_time = time.time()
y_train_predict_stump = stump.predict(X_train)
stump_train_time = time.time() - current_time
stump_train = accuracy_score(y_train, y_train_predict_stump)
stump_train

0.3403637475164298

In [31]:
current_time = time.time()
y_test_predict_stump = stump.predict(X_test)
stump_test_time = time.time() - current_time
stump_test = accuracy_score(y_test, y_test_predict_stump)
stump_test

0.3233511586452763

# Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
depth_range = np.arange(5, 31)
n_trees_range = np.arange(30, 210, 10)
rf_grid = {'max_depth': depth_range, 'n_estimators': n_trees_range}
rf_grid

{'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
        22, 23, 24, 25, 26, 27, 28, 29, 30]),
 'n_estimators': array([ 30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130, 140, 150,
        160, 170, 180, 190, 200])}

In [34]:
rf_gs = GridSearchCV(estimator=RandomForestClassifier(), 
                  param_grid=rf_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [35]:
current_time = time.time()
rf_gs.fit(X_train, y_train)
rf_gs_fit_time = time.time() - current_time

In [36]:
rf_gs.best_params_

{'max_depth': 30, 'n_estimators': 200}

In [37]:
rf_gs.best_score_

0.3937044876332518

In [38]:
best_rf = rf_gs.best_estimator_
best_rf

RandomForestClassifier(max_depth=30, n_estimators=200)

In [39]:
current_time = time.time()
y_train_predict_rf_gs = rf_gs.predict(X_train)
rf_gs_train_time = time.time() - current_time
rf_gs_train = accuracy_score(y_train, y_train_predict_rf_gs)
rf_gs_train

0.9996943298181262

In [40]:
current_time = time.time()
y_test_predict_rf_gs = rf_gs.predict(X_test)
rf_gs_test_time = time.time() - current_time
rf_gs_test = accuracy_score(y_test, y_test_predict_rf_gs)
rf_gs_test

0.40035650623885916

### Fitting, Training, Testing with Best Params

In [41]:
rf = RandomForestClassifier(criterion= 'gini', n_estimators=rf_gs.best_params_['n_estimators'], max_depth=rf_gs.best_params_['max_depth'])

In [42]:
current_time = time.time()
rf.fit(X_train, y_train)
rf_fit_time = time.time() - current_time

In [43]:
current_time = time.time()
y_train_pred_rf = rf.predict(X_train)
rf_train_time = time.time() - current_time
RF_train = accuracy_score(y_train, y_train_pred_rf)
RF_train

0.9996943298181262

In [44]:
current_time = time.time()
y_test_pred_rf = rf.predict(X_test)
rf_test_time = time.time() - current_time
RF_test = accuracy_score(y_test, y_test_pred_rf)
RF_test

0.3932263814616756

# Logistic Regression

In [45]:
# I dont know what parameters to tune

# Linear SVM

In [46]:
from sklearn.svm import LinearSVC

### Finding best params

In [47]:
c_range = np.arange(5.0, 31.0)
lSVM_grid = {'C': c_range}
lSVM_grid

{'C': array([ 5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17.,
        18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30.])}

In [48]:
lSVM_gs = GridSearchCV(estimator=LinearSVC(max_iter=250), 
                  param_grid=lSVM_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [49]:
current_time = time.time()
lSVM_gs.fit(X_train, y_train)
lSVM_gs_fit_time = time.time() - current_time

In [50]:
lSVM_gs.best_params_

{'C': 5.0}

In [51]:
current_time = time.time()
y_pred_train_lSVM_gs = lSVM_gs.predict(X_train)
lSVM_gs_train_time = time.time() - current_time
lSVM_gs_train = accuracy_score(y_train, y_pred_train_lSVM_gs)
lSVM_gs_train

0.33241632278771205

In [52]:
current_time = time.time()
y_pred_test_lSVM_gs = lSVM_gs.predict(X_test)
lSVM_gs_test_time = time.time() - current_time
lSVM_gs_test = accuracy_score(y_test, y_pred_test_lSVM_gs)
lSVM_gs_test

0.33475935828877007

### Fitting, Training, Testing with Best Params

In [53]:
clf = LinearSVC(max_iter = 250, C = lSVM_gs.best_params_['C'])

In [54]:
current_time = time.time()
clf.fit(X_train, y_train)
clf_fit_time = time.time() - current_time

In [55]:
current_time = time.time()
y_pred_train_svm_lin = clf.predict(X_train)
clf_train_time = time.time() - current_time
LinearSVM_train = accuracy_score(y_train, y_pred_train_svm_lin)
LinearSVM_train

0.33241632278771205

In [56]:
current_time = time.time()
y_pred_test_svm_lin = clf.predict(X_test)
clf_test_time = time.time() - current_time
LinearSVM_test = accuracy_score(y_test, y_pred_test_svm_lin)
LinearSVM_test

0.33475935828877007

# Gaussian SVM

In [57]:
from sklearn.svm import SVC

### Finding Best Params

In [58]:
c_range = np.arange(5.0, 31.0)
gamma_range = np.logspace(-9, 3, 13)
gaus_grid = {'C': c_range, 'gamma': gamma_range}
gaus_grid

{'C': array([ 5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17.,
        18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30.]),
 'gamma': array([1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02,
        1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])}

In [59]:
gaus_gs = GridSearchCV(estimator= SVC(max_iter=250, kernel = 'rbf'), 
                  param_grid=gaus_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [60]:
current_time = time.time()
gaus_gs.fit(X_train, y_train)
gaus_gs_fit_time = time.time() - current_time

D:\MiniConda\envs\ai_env\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=250).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [61]:
gaus_gs.best_params_

{'C': 5.0, 'gamma': 100.0}

In [62]:
current_time = time.time()
y_pred_train_gaus_gs = gaus_gs.predict(X_train)
gaus_gs_train_time = time.time() - current_time
gaus_gs_train = accuracy_score(y_train, y_pred_train_gaus_gs)
gaus_gs_train

0.18676448112486627

In [63]:
current_time = time.time()
y_pred_test_gaus_gs = gaus_gs.predict(X_test)
gaus_gs_test_time = time.time() - current_time
gaus_gs_test = accuracy_score(y_test, y_pred_test_gaus_gs)
gaus_gs_test

0.15543672014260249

### Fitting, Training, Testing with Best Params

In [64]:
svm_gaussian = SVC(kernel='rbf', max_iter = 250, C= gaus_gs.best_params_['C'], gamma = gaus_gs.best_params_['gamma'])

In [65]:
current_time = time.time()
svm_gaussian.fit(X_train, y_train)
gaus_fit_time = time.time() - current_time

D:\MiniConda\envs\ai_env\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=250).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [66]:
current_time = time.time()
y_pred_gaussian_svm = svm_gaussian.predict(X_train)
gaus_train_time = time.time() - current_time
GaussianSVM_train = accuracy_score(y_train, y_pred_gaussian_svm)
GaussianSVM_train

0.18676448112486627

In [67]:
current_time = time.time()
y_test_gaussian_svm = svm_gaussian.predict(X_test)
gaus_test_time = time.time() - current_time
GaussianSVM_test = accuracy_score(y_test, y_test_gaussian_svm)
GaussianSVM_test

0.15543672014260249

# K Nearest Neighbors

In [68]:
from sklearn.neighbors import KNeighborsClassifier

### Finding Best Params

In [69]:
n_neighbors_range = np.arange(1, 15)
weights_range = ['uniform', 'distance']
knn_grid = {'n_neighbors': n_neighbors_range, 'weights': weights_range}
knn_grid

{'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 'weights': ['uniform', 'distance']}

In [70]:
knn_gs = GridSearchCV(estimator= KNeighborsClassifier(), 
                  param_grid=knn_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [71]:
current_time = time.time()
knn_gs.fit(X_train, y_train)
knn_gs_fit_time = time.time() - current_time

In [72]:
knn_gs.best_params_

{'n_neighbors': 9, 'weights': 'distance'}

In [73]:
current_time = time.time()
y_pred_train_knn_gs = knn_gs.predict(X_train)
knn_gs_train_time = time.time() - current_time
knn_gs_train = accuracy_score(y_train, y_pred_train_knn_gs)
knn_gs_train

0.9996943298181262

In [74]:
current_time = time.time()
y_pred_test_knn_gs = knn_gs.predict(X_test)
knn_gs_test_time = time.time() - current_time
knn_gs_test = accuracy_score(y_test, y_pred_test_knn_gs)
knn_gs_test

0.2516934046345811

### Fitting, Training, Testing with Best Params

In [75]:
knn = KNeighborsClassifier(n_neighbors= knn_gs.best_params_['n_neighbors'], weights= knn_gs.best_params_['weights'])

In [76]:
current_time = time.time()
knn.fit(X_train, y_train)
KNN_fit_time = time.time() - current_time

In [77]:
current_time = time.time()
y_pred_train_knn = knn.predict(X_train)
KNN_train_time = time.time() - current_time
KNearest_train = accuracy_score(y_train, y_pred_train_knn)
KNearest_train

0.9996943298181262

In [78]:
current_time = time.time()
y_pred_test_knn = knn.predict(X_test)
KNN_test_time = time.time() - current_time
KNearest_test = accuracy_score(y_test, y_pred_test_knn)
KNearest_test

0.2516934046345811

# Adaboost

In [79]:
from sklearn.ensemble import AdaBoostClassifier

### Finding Best Params

In [80]:
n_estimators_range = np.arange(40,60)
#max_depth_range = np.arange(1,7)
ada_grid = {'n_estimators': n_estimators_range}
ada_grid
#, 'max_depth': max_depth_range

{'n_estimators': array([40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
        57, 58, 59])}

In [81]:
max_depth_range = np.arange(1,7)
dt_ada_grid = {'max_depth': max_depth_range}
dt_ada_grid
#, 'max_depth': max_depth_range

{'max_depth': array([1, 2, 3, 4, 5, 6])}

In [82]:
model_est = GridSearchCV(estimator= DecisionTreeClassifier(), 
                  param_grid=dt_ada_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [83]:
model_est.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6])},
             scoring='accuracy')

In [84]:
model_est.best_params_

{'max_depth': 5}

In [85]:
ada_gs = GridSearchCV(estimator= AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = model_est.best_params_['max_depth'])), 
                  param_grid=ada_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [86]:
current_time = time.time()
ada_gs.fit(X_train, y_train)
ada_gs_fit_time = time.time() - current_time

In [87]:
ada_gs.best_params_

{'n_estimators': 44}

In [88]:
current_time = time.time()
y_pred_train_ada_gs = ada_gs.predict(X_train)
ada_gs_train_time = time.time() - current_time
ada_gs_train = accuracy_score(y_train, y_pred_train_ada_gs)
ada_gs_train

0.39828824698150694

In [89]:
current_time = time.time()
y_pred_test_ada_gs = ada_gs.predict(X_test)
ada_gs_test_time = time.time() - current_time
ada_gs_test = accuracy_score(y_test, y_pred_test_ada_gs)
ada_gs_test

0.26773618538324423

### Fitting, Training, Testing with Best Params

In [90]:
adaB = AdaBoostClassifier(DecisionTreeClassifier(max_depth = model_est.best_params_['max_depth']), n_estimators = ada_gs.best_params_['n_estimators'])

In [91]:
current_time = time.time()
adaB.fit(X_train, y_train)
adaB_fit_time = time.time() - current_time

In [92]:
current_time = time.time()
y_pred_train_adaB = adaB.predict(X_train)
adaB_train_time = time.time() - current_time
adaB_train = accuracy_score(y_train, y_pred_train_adaB)
adaB_train

0.38453308879718784

In [93]:
current_time = time.time()
y_pred_test_adaB = adaB.predict(X_test)
adaB_test_time = time.time() - current_time
adaB_test = accuracy_score(y_test, y_pred_test_adaB)
adaB_test

0.25775401069518716

# Results

### Accuracy of each model while finding the best parameter:

In [94]:
print("The accuracy of Decision Tree for the training during hyperameter tuning:  " , dt_gs_train)
print("The accuracy of Decision Tree for the test during hyperameter tuning:  " , dt_gs_test, "\n")

print("The accuracy of Linear SVM for the training during hyperameter tuning:  " , lSVM_gs_train)
print("The accuracy of Linear SVM for the test during hyperameter tuning:  " , lSVM_gs_test, "\n")

print("The accuracy of Gaussian SVM for the training during hyperameter tuning:  " , gaus_gs_train)
print("The accuracy of Gaussian SVM for the test during hyperameter tuning:  " , gaus_gs_train, "\n")

print("The accuracy of K Nearest Neighboors for the training during hyperameter tuning:  " , knn_gs_train)
print("The accuracy of K Nearest Neighboors for the test during hyperameter tuning:  " , knn_gs_test, "\n")

print("The accuracy of Random Forest for the training during hyperameter tuning:  " , rf_gs_train)
print("The accuracy of Random Forest for the test during hyperameter tuning:  " , rf_gs_test, "\n")

print("The accuracy of Adaboost for the training during hyperameter tuning:  " , ada_gs_train)
print("The accuracy of Adaboost for the test during hyperameter tuning:  " , ada_gs_test, "\n")

The accuracy of Decision Tree for the training during hyperameter tuning:   0.3403637475164298
The accuracy of Decision Tree for the test during hyperameter tuning:   0.3226381461675579 

The accuracy of Linear SVM for the training during hyperameter tuning:   0.33241632278771205
The accuracy of Linear SVM for the test during hyperameter tuning:   0.33475935828877007 

The accuracy of Gaussian SVM for the training during hyperameter tuning:   0.18676448112486627
The accuracy of Gaussian SVM for the test during hyperameter tuning:   0.18676448112486627 

The accuracy of K Nearest Neighboors for the training during hyperameter tuning:   0.9996943298181262
The accuracy of K Nearest Neighboors for the test during hyperameter tuning:   0.2516934046345811 

The accuracy of Random Forest for the training during hyperameter tuning:   0.9996943298181262
The accuracy of Random Forest for the test during hyperameter tuning:   0.40035650623885916 

The accuracy of Adaboost for the training during 

### Time taken for each model during hyperameter tuning:

In [95]:
print("The number of seconds for fitting in Decision Tree during hyperameter tuning:  " , dt_gs_fit_time)
print("The number of seconds for testing the train data in Decision Tree during hyperameter tuning:  " , dt_gs_train_time)
print("The number of seconds for testing the test data in Decision Tree during hyperameter tuning:  " , dt_gs_test_time, "\n")

print("The number of seconds for fitting in Linear SVM during hyperameter tuning:  " , lSVM_gs_fit_time)
print("The number of seconds for testing the train data in Linear SVM during hyperameter tuning:  " , lSVM_gs_train_time)
print("The number of seconds for testing the test data in Linear SVM during hyperameter tuning:  " , lSVM_gs_test_time, "\n")

print("The number of seconds for fitting in Gaussian SVM during hyperameter tuning:  " , gaus_gs_fit_time)
print("The number of seconds for testing the train data in Gaussian SVM during hyperameter tuning:  " , gaus_gs_train_time)
print("The number of seconds for testing the test data in Gaussian SVM during hyperameter tuning:  " , gaus_gs_test_time, "\n")

print("The number of seconds for fitting in K Nearest Neighboors during hyperameter tuning:  " , knn_gs_fit_time)
print("The number of seconds for testing the train data in K Nearest Neighboors during hyperameter tuning:  " , knn_gs_train_time)
print("The number of seconds for testing the test data in K Nearest Neighboors during hyperameter tuning:  " , knn_gs_test_time, "\n")

print("The number of seconds for fitting in Random Forest during hyperameter tuning:  " , rf_gs_fit_time)
print("The number of seconds for testing the train data in Random Forest during hyperameter tuning:  " , rf_gs_train_time)
print("The number of seconds for testing the test data in Random Forest during hyperameter tuning:  " , rf_gs_test_time, "\n")

print("The number of seconds for fitting in Adaboost during hyperameter tuning:  " , ada_gs_fit_time)
print("The number of seconds for testing the train data in Adaboost during hyperameter tuning:  " , ada_gs_train_time)
print("The number of seconds for testing the test data in Adaboost during hyperameter tuning:  " , ada_gs_test_time)

The number of seconds for fitting in Decision Tree during hyperameter tuning:   50.258026361465454
The number of seconds for testing the train data in Decision Tree during hyperameter tuning:   0.00498652458190918
The number of seconds for testing the test data in Decision Tree during hyperameter tuning:   0.002991914749145508 

The number of seconds for fitting in Linear SVM during hyperameter tuning:   203.024094581604
The number of seconds for testing the train data in Linear SVM during hyperameter tuning:   0.0059854984283447266
The number of seconds for testing the test data in Linear SVM during hyperameter tuning:   0.0029931068420410156 

The number of seconds for fitting in Gaussian SVM during hyperameter tuning:   1602.1033318042755
The number of seconds for testing the train data in Gaussian SVM during hyperameter tuning:   6.32808518409729
The number of seconds for testing the test data in Gaussian SVM during hyperameter tuning:   2.7656500339508057 

The number of seconds f

### Accuracy of the models with the best parameters:

In [96]:
print("The accuracy of Decision Tree for the training with the best parameters:  " , stump_train)
print("The accuracy of Decision Tree for the test with the best parameters:  " , stump_test, "\n")

print("The accuracy of Linear SVM for the training with the best parameters:  " , LinearSVM_train)
print("The accuracy of Linear SVM for the test with the best parameters:  " , LinearSVM_test, "\n")

print("The accuracy of Gaussian SVM for the training with the best parameters:  " , GaussianSVM_train)
print("The accuracy of Gaussian SVM for the test with the best parameters:  " , GaussianSVM_train, "\n")

print("The accuracy of K Nearest Neighboors for the training with the best parameters:  " , KNearest_train)
print("The accuracy of K Nearest Neighboors for the test with the best parameters:  " , KNearest_test, "\n")

print("The accuracy of Random Forest for the training with the best parameters:  " , RF_train)
print("The accuracy of Random Forest for the test with the best parameters:  " , RF_test, "\n")

print("The accuracy of Adaboost for the training with the best parameters:  " , adaB_train)
print("The accuracy of Adaboost for the test with the best parameters:  " , adaB_test, "\n")

The accuracy of Decision Tree for the training with the best parameters:   0.3403637475164298
The accuracy of Decision Tree for the test with the best parameters:   0.3233511586452763 

The accuracy of Linear SVM for the training with the best parameters:   0.33241632278771205
The accuracy of Linear SVM for the test with the best parameters:   0.33475935828877007 

The accuracy of Gaussian SVM for the training with the best parameters:   0.18676448112486627
The accuracy of Gaussian SVM for the test with the best parameters:   0.18676448112486627 

The accuracy of K Nearest Neighboors for the training with the best parameters:   0.9996943298181262
The accuracy of K Nearest Neighboors for the test with the best parameters:   0.2516934046345811 

The accuracy of Random Forest for the training with the best parameters:   0.9996943298181262
The accuracy of Random Forest for the test with the best parameters:   0.3932263814616756 

The accuracy of Adaboost for the training with the best para

### Time taken for each model with the best parameters

In [97]:
print("The number of seconds for fitting in Decision Tree with the best parameters:  " , stump_fit_time)
print("The number of seconds for testing the train data in Decision Tree with the best parameters:  " , stump_train_time)
print("The number of seconds for testing the test data in Decision Tree with the best parameters:  " , stump_test_time, "\n")

print("The number of seconds for fitting in Linear SVM with the best parameters:  " , clf_fit_time)
print("The number of seconds for testing the train data in Linear SVM with the best parameters:  " , clf_train_time)
print("The number of seconds for testing the test data in Linear SVM with the best parameters:  " , clf_test_time, "\n")

print("The number of seconds for fitting in Gaussian SVM with the best parameters:  " , gaus_fit_time)
print("The number of seconds for testing the train data in Gaussian SVM with the best parameters:  " , gaus_train_time)
print("The number of seconds for testing the test data in Gaussian SVM with the best parameters:  " , gaus_test_time, "\n")

print("The number of seconds for fitting in K Nearest Neighboors with the best parameters:  " , KNN_fit_time)
print("The number of seconds for testing the train data in K Nearest Neighboors with the best parameters:  " , KNN_train_time)
print("The number of seconds for testing the test data in K Nearest Neighboors with the best parameters:  " , KNN_test_time, "\n")

print("The number of seconds for fitting in Random Forest with the best parameters:  " , rf_fit_time)
print("The number of seconds for testing the train data in Random Forest with the best parameters:  " , rf_train_time)
print("The number of seconds for testing the test data in Random Forest with the best parameters:  " , rf_test_time, "\n")

print("The number of seconds for fitting in Adaboost with the best parameters:  " , adaB_fit_time)
print("The number of seconds for testing the train data in Adaboost with the best parameters:  " , adaB_train_time)
print("The number of seconds for testing the test data in Adaboost with the best parameters:  " , adaB_test_time)

The number of seconds for fitting in Decision Tree with the best parameters:   0.2493593692779541
The number of seconds for testing the train data in Decision Tree with the best parameters:   0.00598907470703125
The number of seconds for testing the test data in Decision Tree with the best parameters:   0.0029871463775634766 

The number of seconds for fitting in Linear SVM with the best parameters:   2.833442449569702
The number of seconds for testing the train data in Linear SVM with the best parameters:   0.003988504409790039
The number of seconds for testing the test data in Linear SVM with the best parameters:   0.003002643585205078 

The number of seconds for fitting in Gaussian SVM with the best parameters:   2.2385144233703613
The number of seconds for testing the train data in Gaussian SVM with the best parameters:   5.9361186027526855
The number of seconds for testing the test data in Gaussian SVM with the best parameters:   2.5731446743011475 

The number of seconds for fitt

# Conclusions:

# Observation


# Conclusion


In [98]:
resultsTrain = pd.DataFrame.from_dict({"Adaboost": adaB_train, "Decision Tree" : stump_train, "Linear SVM" : LinearSVM_train, "Gaussian SVM" : GaussianSVM_train, "K Nearest" : KNearest_train, "Random Forest" : RF_train}, orient= 'index')
resultsTrain

,0
Adaboost,0.384533
Decision Tree,0.340364
Linear SVM,0.332416
Gaussian SVM,0.186764
K Nearest,0.999694
Random Forest,0.999694


In [99]:
resultsTest = pd.DataFrame.from_dict({"Adaboost": ada_gs_test, "Decision Tree" : stump_test, "Linear SVM" : LinearSVM_test, "Gaussian SVM" : GaussianSVM_test, "K Nearest" : KNearest_test, "Random Forest" : RF_test}, orient= 'index')
resultsTest

,0
Adaboost,0.267736
Decision Tree,0.323351
Linear SVM,0.334759
Gaussian SVM,0.155437
K Nearest,0.251693
Random Forest,0.393226


In [100]:
resultsTrain.to_csv("C:\\Users\\samue\\Desktop\\AI_Proj\\bigHyperTuningTrain.csv")
resultsTest.to_csv("C:\\Users\\samue\\Desktop\\AI_Proj\\bigHyperTuningTest.csv")